In [3]:
import os; os.chdir('/Users/sqian/MKTSRV')
import pandas as pd; pd.set_option('mode.chained_assignment',None)
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import datetime
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy
import re
import xlrd
from datetime import date
from xlsxwriter.utility import xl_rowcol_to_cell
from random import sample
from math import log10, floor
from src.data_models.smartshelper import metrics, fixNum, unSMART, cparty, littleQA, totalUP
from src.data_models.SmartsDataModel import SmartsDataModel
from src.utility.DataModelUtility import execute_query_data_frame

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
alerting_1 = datetime.date(2019,3,1)
alerting_2 = datetime.date(2019,4,30)
acc = 97280

In [7]:
query = """
        SELECT *
        from account_derived_metadata
        where exchange_account_id in ({})
        """.format(acc)

test = execute_query_data_frame(query,'engine')
test.transpose()

ssh_connect 0:00:00.791046


,0
exchange_account_id,811415
user_or_account_name,HRTJ Limited
name_source,Account Name
country_code,ky
state_code,None
is_institutional,True
first_verified_at,2018-03-07 19:01:35.812000
is_active,True
last_modified,2018-10-03 21:12:56.561000
account_group_id,NaN


In [8]:
query = """
        SELECT *
        from order_fill_event
        where account_id in ({}) and created::date between '2019-01-01' and '2019-04-04'
        """.format(acc)
test = execute_query_data_frame(query,'engine')
test2 = pd.pivot(test.groupby(['account_id','trading_pair','side']).sum()['quantity'].reset_index(), index='trading_pair', columns='side',values='quantity'); test2

ssh_connect 0:00:00.803397


side,buy,sell
trading_pair,,
BTCUSD,14110.662419,14133.426449
ETHBTC,18031.537063,17781.537233
ETHUSD,103157.480882,102932.807274


In [9]:
query3 = """ 
        SELECT account_id, trading_pair, firsttable.side, sum(firsttable.price * COALESCE(secondtable.open_price,1) * quantity) as val, sum(quantity) as units
        
        from  
            (SELECT account_id, event_id, side, price, quantity, trading_pair, date_trunc('minute', created) start_time, substring(trading_pair::varchar,4,6) as trunc
            from order_fill_event 
            where created::date between '{t1}' and '{t2}' and account_id in ({x})) as firsttable
        left join 
            (SELECT substring(trading_pair::varchar,1,3) as truncmatch, start_time, open_price
            from candles_1m 
            where start_time::date between '{t1}' and '{t2}' and substring(trading_pair::varchar,4,6) = 'USD') as secondtable
        
        on firsttable.trunc = secondtable.truncmatch and firsttable.start_time = secondtable.start_time

        group by firsttable.trading_pair, firsttable.side, account_id;
        """.format(t1 = alerting_1.strftime("%Y%m%d"), t2 = alerting_2.strftime("%Y%m%d"), x=acc)

In [38]:
output3_D = execute_query_data_frame(query3,'engine'); output3_D
output3 = pd.pivot_table(output3_D, values='val', index=['trading_pair'], columns=['side'], aggfunc=np.sum)
output3['net'] = output3['sell'] - output3['buy']
output3a = output3.sort_values('sell')
pd.concat([output3a,pd.DataFrame(output3a.sum(axis=0)).rename(columns={0:'TOTAL'}).transpose()], axis=0).astype(int)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4p1)
INFO:paramiko.transport:Authentication (publickey) successful!


ssh_connect 0:00:00.719842


INFO:root:Loaded dataframe with 18 rows in 0:00:03.575946.


side,buy,sell,net
ZECBTC,661049,576713,-84336
LTCBTC,713122,749856,36734
ETHBTC,1380426,1036528,-343897
BCHBTC,1714891,1600834,-114057
ZECUSD,1928784,1962964,34179
BCHUSD,5407557,5503341,95783
LTCUSD,8967901,8991877,23975
ETHUSD,13827897,14242645,414747
BTCUSD,35251129,34909442,-341687
TOTAL,69852759,69574202,-278556
